Modelados de datos para funciónes de Machine Learning

Importamos las librerías 

In [1]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.metrics.pairwise import cosine_similarity
import operator
import fastparquet as fp 
import pyarrow as pa
import pyarrow.parquet as pq
import google.cloud.storage as storage
import scipy.stats as stats

Modelo de recomendación item_item 

El modelado de recomendación item-item es una técnica en sistemas de recomendación que se centra en las similitudes entre los elementos: 
1- Matriz de interacciones usuario-elemento
2- Medición de similitud entre elementos
3- Cálculo de puntajes de recomendación
4- Selección de elementos recomendados
5-Optimización y ajuste
6- Evaluación del modelo

In [2]:
modelo_item = pd.read_parquet("Data/modelo_item.parquet")
modelo_item.head(5)

,item_id,app_name,genres
0,761140,Lost Summoner Kitty,Action
1,643980,Ironbound,Free to Play
2,670290,Real Pool 3D - Poolians,Casual
3,767400,弹炸人2222,Action
4,772540,Battle Royale Trainer,Action


Creamos un Datafarme que incluya en sus columnas los géneros que puedan ser conbinados con los nombres de los juegos, 
en cada uno las de las filas 

In [4]:
#Creamos los dummies
modelo_item = pd.get_dummies(modelo_item, columns=["genres"], prefix="")

modelo_item = modelo_item.groupby(["item_id","app_name"]).sum().reset_index()

modelo_item.head(5)

,item_id,app_name,_Accounting,_Action,_Adventure,_Animation &amp; Modeling,_Audio Production,_Casual,_Design &amp; Illustration,_Early Access,...,_Photo Editing,_RPG,_Racing,_Simulation,_Software Training,_Sports,_Strategy,_Utilities,_Video Production,_Web Publishing
0,10,Counter-Strike,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,20,Team Fortress Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,30,Day of Defeat,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,40,Deathmatch Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,50,Half-Life: Opposing Force,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Calculamos la similitud del Coseno 



In [5]:
# Crea dos vectores
v1 = [1, 2, 3]
v2 = [4, 5, 6]

# Calcula la similitud coseno
cos_sim = cosine_similarity([v1, v2])

In [6]:

cos_sim.shape

(2, 2)

In [7]:

# Imprime el resultado
print(cos_sim)

[[1.         0.97463185]
 [0.97463185 1.        ]]


In [8]:
data = pd.read_parquet('Data\steam_games.parquet')



In [9]:
print(data)

                       app_name  release_date  \
0           Lost Summoner Kitty           NaN   
1                     Ironbound           NaN   
2       Real Pool 3D - Poolians           NaN   
3                       弹炸人2222           NaN   
4         Battle Royale Trainer           NaN   
...                         ...           ...   
28894              Kebab it Up!           NaN   
28895            Colony On Mars           NaN   
28896  LOGistICAL: South Africa           NaN   
28897             Russian Roads           NaN   
28898       EXIT 2 - Directions           NaN   

                                                    tags  price  item_id  \
0      ['Strategy', 'Action', 'Indie', 'Casual', 'Sim...   4.99   761140   
1      ['Free to Play', 'Strategy', 'Indie', 'RPG', '...   0.00   643980   
2      ['Free to Play', 'Simulation', 'Sports', 'Casu...   0.00   670290   
3                      ['Action', 'Adventure', 'Casual']   0.99   767400   
4      ['Action', 'Adventure', 

In [10]:
def recomendacion_juego(id):
    
    id = int(id)
    # Filtrar el juego e igualarlo a  su ID
    juego_seleccionado = modelo_item[modelo_item['item_id'] == id]
    # devolver error en caso de vacio
    if juego_seleccionado.empty:
        return "El juego con el ID especificado no existe en la base de datos."
    
    # Calcula la similitud del juego que se ingresa con otros juegos del dataframe
    similarity_scores = cos_sim[modelo_item[modelo_item['item_id'] == id].index[0]]

     # Calcula los índices de los juegos más similares (excluyendo el juego de entrada)
    indices_juegos_similares = similarity_scores.argsort()[::-1][1:6]
    
    # Obtener los nombres de los juegos 5 recomendados
    juegos_recomendados = modelo_item.iloc[indices_juegos_similares]['app_name']
    
    return juegos_recomendados
    
    

In [11]:
recomendacion_juego( 10)

1    Team Fortress Classic
Name: app_name, dtype: object

Modelo para Render

In [12]:
modelo_item

,item_id,app_name,_Accounting,_Action,_Adventure,_Animation &amp; Modeling,_Audio Production,_Casual,_Design &amp; Illustration,_Early Access,...,_Photo Editing,_RPG,_Racing,_Simulation,_Software Training,_Sports,_Strategy,_Utilities,_Video Production,_Web Publishing
0,10,Counter-Strike,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,20,Team Fortress Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,30,Day of Defeat,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,40,Deathmatch Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,50,Half-Life: Opposing Force,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28894,2028055,Tom Clancy's Ghost Recon Future Soldier - Seas...,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28895,2028056,Worms Revolution Season Pass,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
28896,2028062,Call of Duty®: Black Ops II Season Pass,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28897,2028103,Assassin’s Creed® III Season Pass,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Dividimos el DataFarme por la mitad con el fin de lograr el modelo en render

In [13]:
#Cuento filas
cant_filas= len(modelo_item)

#Calculo la mitad
mitad_filas= cant_filas // 10
 
#Selecciono la mitad superior
modelo_render= modelo_item.iloc[:mitad_filas]

In [14]:
modelo_render.shape

(2889, 25)

In [15]:
similitudes_render = cosine_similarity(modelo_render.iloc[:,3:])

Guardamos el Datafarme

In [16]:
modelo_render.to_parquet("Data/modelo_render.parquet")

Creamos una nueva función 

In [17]:
def encontrar_juegos_similares(id):
    # Encuentra el índice del juego ingresado por ID
    juego_indice = modelo_render.index[modelo_render['item_id'] == id].tolist()[0]
    
    # Extrae las características del juego ingresado
    juego_caracteristicas = modelo_render.iloc[juego_indice, 3:].values.reshape(1, -1)
    
    # Calcula la similitud coseno entre el juego ingresado y todos los demás juegos
    similitudes_render = cosine_similarity(modelo_render.iloc[:, 3:], juego_caracteristicas)
    
    # Obtiene los índices de los juegos más similares (excluyendo el juego de entrada)
    indices_juegos_similares = similitudes_render.argsort(axis=0)[::-1][1:6]
    indices_juegos_similares = indices_juegos_similares.flatten()[1:]
    
    # Obtiene los juegos más similares en función de los índices
    juegos_similares = modelo_render.iloc[indices_juegos_similares]['app_name']
    
    return juegos_similares


In [18]:
encontrar_juegos_similares(3310)

949      Westward Collection
950    Cake Mania Collection
951             Monster Mash
952                    Kuros
Name: app_name, dtype: object

Función acotada(no se utiliza)

In [19]:
def recomendacion_juego(id):
    
    id = int(id)
    # Filtrar el juego e igualarlo a  su ID
    juego = modelo_render[modelo_render['item_id'] == id]
    # devolver error en caso de vacio
    if juego.empty:
        return "El juego con el ID especificado no existe en la base de datos."
    

    
    # Calcular la matriz de similitud coseno
    similitudes_render = cosine_similarity(modelo_render.iloc[:,3:])
    
    # Calcula la similitud del juego que se ingresa con otros juegos del dataframe
    similarity_scores = similitudes_render[modelo_render[modelo_render['item_id'] == id].index[0]]
    
    # Calcula los índices de los juegos más similares (excluyendo el juego de entrada)
    indices_juegos_similares = similarity_scores.argsort()[::-1][1:6]
    
    # Obtener los nombres de los juegos 5 recomendados
    juegos_recomendados = modelo_render.iloc[indices_juegos_similares]['app_name']
    #Obtengo un diccionario para la salida en la api
    #juegos_recomendados=juegos_recomendados.to_dict()
    return {
        "Juegos_recomendados" :juegos_recomendados}

In [20]:
recomendacion_juego(3310)

{'Juegos_recomendados': 975        Mahjong Roadshow™
 949      Westward Collection
 950    Cake Mania Collection
 951             Monster Mash
 952                    Kuros
 Name: app_name, dtype: object}

In [21]:
def recommend_games(id: int):
    '''
    Esta función recomienda 5 juegos a partir del juego ingresado.

    Args:
        game_id (int): ID único del videojuego al cual se le harán las recomendaciones.
    '''
    # Verifica si el juego con game_id existe en df_games
    game = modelo_render[modelo_render['item_id'] == id]

    if game.empty:
        return("El juego '{id}' no posee registros.")
    
    # Obtiene el índice del juego dado
    idx = game.index[0]

    # Toma una muestra aleatoria del DataFrame df_games
    sample_size = 2000  # Define el tamaño de la muestra (ajusta según sea necesario)
    df_sample = modelo_render.sample(n=sample_size, random_state=42)  # Ajusta la semilla aleatoria según sea necesario

    # Calcula la similitud de contenido solo para el juego dado y la muestra
    sim_scores = cosine_similarity([modelo_render.iloc[idx, 3:]], df_sample.iloc[:, 3:])

    # Obtiene las puntuaciones de similitud del juego dado con otros juegos
    sim_scores = sim_scores[0]

    # Ordena los juegos por similitud en orden descendente
    similar_games = [(i, sim_scores[i]) for i in range(len(sim_scores)) if i != idx]
    similar_games = sorted(similar_games, key=lambda x: x[1], reverse=True)

    # Obtiene los 5 juegos más similares
    similar_game_indices = [i[0] for i in similar_games[:5]]

    # Lista de juegos similares (solo nombres)
    similar_game_names = df_sample['app_name'].iloc[similar_game_indices].tolist()

    return {"similar_games": similar_game_names}

In [22]:
recommend_games(10)

{'similar_games': ['Tomb Raider: Shanty Town',
  'Magicka: Holiday Spirit Item Pack',
  'Nuclear Dawn',
  'Sam & Max 104: Abe Lincoln Must Die!',
  'Shadowgrounds Survivor']}